# Web Scraping Tutorial

This tutorial will teach you how to use Python to scrap and extract data from a web page. We will use two packages, `requests` to scrap the webpage and `BeautifulSoup` to extract the data.

Many good references on web scraping are available online. I would recommend the following resources:
1. Automate Boring Stuff with Python by Al Sweigart (2020) has a chapter on Web Scraping tutorial, which can be read [online](https://automatetheboringstuff.com/2e/chapter12/).
2. Web Scraping With Python by Ryan Mitchell (2018) is a bit old book but provides a comprehensive guide to the topic.

## Step 0: Getting to know the web page

In this tutorial, we will try to extract the cryptocurrency market prices from the CoinGecko website https://www.coingecko.com/.

Your first step should always be to familiarize yourself with the website you want to scrape. Take a look at the website and try to inspect the HTML elements on the webpage.

## Step 1: Scrap a web page

Now, we are ready to scrap a webpage we want to get the data from with the `requests` package. We will use the following functions:

* `requests.get('URL')` - make a request to the specified URL
* `r.status_code` - get the status code of the request
* `r.content` - get the binary content of the page

More functions in the `requests` package are available in [its documentation](https://requests.readthedocs.io/en/latest/).

In [ ]:
# First, we will import the requests package
import requests

In [ ]:
# Request the webpage


In [ ]:
# Type of the request we've got


In [ ]:
# Check the status code


In [ ]:
# Get the header of the web page


In [ ]:
# Get the content of the web page


In [ ]:
# Save the content of web page to the local drive


## Step 2: Extract data from the web page

After we crawled the web page and download it to the local disk, we will use `BeautifulSoup` package to parse HTML file and access the content. We will use the following functions:

**1. Load the web page to BeautifulSoup**
* `soup = BeautifulSoup(html_doc, 'html.parser')` - parse the HTML content to BeautifulSoup object

**2. Get the content of the element**
* `soup.title` - get the title of the page
* `soup.title.string` - get the string in the title tag
* `soup.h1` - get the H1 element in the web page
* `soup.h1.attrs` - get all attributes in the H1 element
* `soup.h1['class']` - get the class attribute in the H1 element

**3. Look for the element in the web page**
* `soup.find('HTML_tag')` - get the element from an HTML tag
* `soup.find_all('HTML_tag')` - get the list of elelemts that has the specified HTML tag
* `soup.select('CSS_selector')` - get the list of elements with the specified [CSS selector](https://www.w3schools.com/cssref/css_selectors.asp)

In [ ]:
# First, we will import the BeautifulSoup from bs4 package
from bs4 import BeautifulSoup

In [ ]:
# Load the web page and parse it to BeautifulSoup


In [ ]:
# Check the type of our soup object


In [ ]:
# Get all text in the web page


In [ ]:
# Get the title of the page


In [ ]:
# We can also get the page title using soup.find() function


In [ ]:
# Other HTML tags also work too


Now, we will extract the cryptocurrencies market price from the table.

In [ ]:
# Get the table element in the web page


In [ ]:
# Get the table headers


In [ ]:
# If there are > 1 elements that match the tagged, 
# use soup.find_all() to retrieve all of them as a list.


In [ ]:
# Iterate over rows and get the data for each coin


## Step 3: Create data table and save as CSV file

Let's wrap our data table as the pandas's DataFrame and save it as a CSV file.

In [ ]:
import pandas as pd